# 🚗 Car Knowledge Assistant

## Overview

This automotive knowledge assistant uses AI-powered web search with a **3-expert verification system** to provide reliable information about:
- **Maintenance & Repairs**: Service schedules, common issues, troubleshooting
- **Interior Customization**: Upgrades, modifications, accessories
- **Features & Controls**: How-to guides, feature explanations
- **Manual Information**: Owner's manual details, specifications
- **DIY Guides**: Step-by-step repair and maintenance tutorials

### How It Works (Unique 3-Expert System)

1. **Expert 1 (Primary Mechanic)**: Researches and provides initial answer
2. **Expert 2 (Verification Specialist)**: Cross-checks Expert 1's information
3. **Expert 3 (Conflict Resolver)**: Reviews both experts and resolves any conflicts
4. **Beautiful Dashboard**: Results displayed with verification status

---

**✨ Colab-Optimized**: Works perfectly in Google Colab! Verified information you can trust!

## Installation

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-python langchain-tavily wikipedia

## Setup API Keys

In [2]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

OPENAI_API_KEY: ··········
TAVILY_API_KEY: ··········


## Import Dependencies

In [3]:
from typing import List, Annotated
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, get_buffer_string
from langchain_tavily import TavilySearch
from langchain_community.document_loaders import WikipediaLoader
from IPython.display import display, HTML
import operator

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Initialize Tavily Search
tavily_search = TavilySearch(max_results=3)

print("✅ All dependencies loaded successfully!")

✅ All dependencies loaded successfully!


## Define Expert System Models

In [4]:
class CarExpert(BaseModel):
    """Automotive expert"""
    name: str = Field(description="Name of the automotive expert")
    role: str = Field(description="Expert role (Primary Mechanic / Verification Specialist / Conflict Resolver)")
    specialty: str = Field(description="Area of automotive expertise")

    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nSpecialty: {self.specialty}\n"

class ExpertResponse(BaseModel):
    """Expert's response with confidence and sources"""
    expert_name: str = Field(description="Name of the expert")
    response: str = Field(description="Detailed response")
    confidence: str = Field(description="Confidence level: High/Medium/Low")
    sources: List[str] = Field(description="List of source URLs")
    issues_found: List[str] = Field(default=[], description="Issues or conflicts found (for verification)")

class SearchQuery(BaseModel):
    search_query: str = Field(description="Automotive search query")

class VerificationState(TypedDict):
    car_topic: str
    expert1_response: ExpertResponse
    expert2_response: ExpertResponse
    expert3_response: ExpertResponse
    search_context: list
    final_answer: str

print("✅ Expert system models defined!")

✅ Expert system models defined!


## Web Search Functions

In [5]:
def search_automotive_info(query: str) -> str:
    """Search for automotive information"""
    print(f"      🔍 Searching for: {query}")

    try:
        search_results = tavily_search.invoke(query)

        if isinstance(search_results, list):
            search_docs = search_results
        elif isinstance(search_results, dict):
            search_docs = search_results.get("results", [])
        else:
            search_docs = []

        if search_docs:
            # Limit content - 500 chars per doc, max 3 docs
            formatted_docs = "\n\n---\n\n".join([
                f'<Source url="{doc.get("url", "N/A")}"/>\n{str(doc.get("content", doc.get("snippet", "")))[:500]}...\n</Source>'
                for doc in search_docs[:3]
                if isinstance(doc, dict)
            ])
            print(f"      ✅ Found {len(search_docs)} sources")
            return formatted_docs
        else:
            print("      ⚠️ No results found")
            return "No search results found."

    except Exception as e:
        print(f"      ❌ Search error: {e}")
        return f"Search error: {str(e)}"

print("✅ Search functions ready!")

✅ Search functions ready!


## Expert 1: Primary Mechanic (Information Provider)

In [6]:
expert1_instructions = """You are a PRIMARY AUTOMOTIVE MECHANIC providing initial information.

Topic: {topic}

Research Context:
{context}

Your job:
1. Provide detailed, accurate automotive information
2. Base all answers on the provided sources
3. Include:
   - Step-by-step instructions (if applicable)
   - Safety warnings
   - Required tools/parts
   - Common mistakes to avoid
4. Cite sources using [1], [2], etc.
5. Rate your confidence: High/Medium/Low

Be thorough and practical. This is the initial answer that will be verified."""

def expert1_provide_info(state: VerificationState):
    """Expert 1 provides initial answer"""
    topic = state['car_topic']

    print("\n🔧 Expert 1 (Primary Mechanic) researching...")

    # Search for information
    context = search_automotive_info(topic)

    # Generate response
    system_message = expert1_instructions.format(topic=topic, context=context)

    structured_llm = llm.with_structured_output(ExpertResponse)
    response = structured_llm.invoke([
        SystemMessage(content=system_message),
        HumanMessage(content=f"Provide comprehensive information about: {topic}")
    ])

    print(f"   ✅ Expert 1 complete (Confidence: {response.confidence})")

    return {
        "expert1_response": response,
        "search_context": [context]
    }

print("✅ Expert 1 (Primary Mechanic) ready!")

✅ Expert 1 (Primary Mechanic) ready!


## Expert 2: Verification Specialist (Cross-Checker)

In [7]:
expert2_instructions = """You are a VERIFICATION SPECIALIST cross-checking automotive information.

Topic: {topic}

Expert 1's Answer:
{expert1_answer}

Additional Research Context:
{context}

Your job:
1. Verify Expert 1's information against sources
2. Check for:
   - Factual accuracy
   - Safety concerns
   - Missing critical steps
   - Incorrect specifications
3. If you find issues, list them clearly
4. Provide additional information Expert 1 missed
5. Cite your sources [1], [2], etc.
6. Rate confidence: High/Medium/Low

Be thorough - lives depend on accurate car maintenance!"""

def expert2_verify_info(state: VerificationState):
    """Expert 2 verifies and cross-checks"""
    topic = state['car_topic']
    expert1 = state['expert1_response']

    print("\n✅ Expert 2 (Verification Specialist) cross-checking...")

    # Search for verification
    verify_query = f"{topic} verification safety check correct procedure"
    context = search_automotive_info(verify_query)

    # Verify response
    system_message = expert2_instructions.format(
        topic=topic,
        expert1_answer=expert1.response,
        context=context
    )

    structured_llm = llm.with_structured_output(ExpertResponse)
    response = structured_llm.invoke([
        SystemMessage(content=system_message),
        HumanMessage(content=f"Verify and cross-check the information about: {topic}")
    ])

    print(f"   ✅ Expert 2 complete (Confidence: {response.confidence})")
    if response.issues_found:
        print(f"   ⚠️ Issues found: {len(response.issues_found)}")

    return {
        "expert2_response": response,
        "search_context": state['search_context'] + [context]
    }

print("✅ Expert 2 (Verification Specialist) ready!")

✅ Expert 2 (Verification Specialist) ready!


## Expert 3: Conflict Resolver (Final Authority)

In [8]:
expert3_instructions = """You are a SENIOR AUTOMOTIVE EXPERT and CONFLICT RESOLVER.

Topic: {topic}

Expert 1's Response (Primary Mechanic):
{expert1_answer}
Confidence: {expert1_confidence}

Expert 2's Verification:
{expert2_answer}
Issues Found: {expert2_issues}
Confidence: {expert2_confidence}

All Research Sources:
{context}

Your job:
1. Review both experts' responses
2. Resolve any conflicts or discrepancies
3. Create the FINAL, AUTHORITATIVE answer
4. If conflicts exist, explain which expert is correct and why
5. Integrate the best information from both experts
6. Add any critical information both missed
7. Ensure safety is prioritized
8. Provide clear, actionable guidance
9. Include all relevant sources

Your answer will be the final word - make it comprehensive and accurate!"""

def expert3_resolve_conflicts(state: VerificationState):
    """Expert 3 resolves conflicts and provides final answer"""
    topic = state['car_topic']
    expert1 = state['expert1_response']
    expert2 = state['expert2_response']
    contexts = state['search_context']

    print("\n🏆 Expert 3 (Conflict Resolver) reviewing...")

    # Compile all context
    all_context = "\n\n".join(contexts)

    # Resolve and finalize
    system_message = expert3_instructions.format(
        topic=topic,
        expert1_answer=expert1.response,
        expert1_confidence=expert1.confidence,
        expert2_answer=expert2.response,
        expert2_issues=expert2.issues_found,
        expert2_confidence=expert2.confidence,
        context=all_context[:3000]  # Limit context
    )

    structured_llm = llm.with_structured_output(ExpertResponse)
    response = structured_llm.invoke([
        SystemMessage(content=system_message),
        HumanMessage(content=f"Provide the final, authoritative answer about: {topic}")
    ])

    print(f"   ✅ Expert 3 complete (Final Confidence: {response.confidence})")

    return {
        "expert3_response": response,
        "final_answer": response.response
    }

print("✅ Expert 3 (Conflict Resolver) ready!")

✅ Expert 3 (Conflict Resolver) ready!


## Build Verification Graph

In [9]:
# Build the 3-expert verification workflow
workflow = StateGraph(VerificationState)

# Add expert nodes
workflow.add_node("expert1_provide", expert1_provide_info)
workflow.add_node("expert2_verify", expert2_verify_info)
workflow.add_node("expert3_resolve", expert3_resolve_conflicts)

# Define workflow: Expert 1 → Expert 2 → Expert 3
workflow.add_edge(START, "expert1_provide")
workflow.add_edge("expert1_provide", "expert2_verify")
workflow.add_edge("expert2_verify", "expert3_resolve")
workflow.add_edge("expert3_resolve", END)

# Compile
verification_graph = workflow.compile()

print("✅ 3-Expert Verification System ready!")
print("   Flow: Primary Mechanic → Verification → Conflict Resolution")

✅ 3-Expert Verification System ready!
   Flow: Primary Mechanic → Verification → Conflict Resolution


## 🎨 Beautiful Dashboard Display

In [24]:
# 📝 Markdown Display Function for Car Knowledge Assistant
# Copy and paste this cell to replace the HTML dashboard

from IPython.display import display, Markdown

def create_car_dashboard(topic, expert1, expert2, expert3):
    """
    Creates clean Markdown display showing 3-expert verification process
    """

    # Determine overall status
    has_conflicts = len(expert2.issues_found) > 0
    status = "⚠️ CONFLICTS RESOLVED" if has_conflicts else "✅ VERIFIED"

    # Build markdown report
    markdown_report = f"""
# 🚗 Car Knowledge Assistant

## {topic}

**Status:** {status}

---

## 🔧 Expert 1: Primary Mechanic

**Confidence:** {expert1.confidence} {'🟢' if expert1.confidence == 'High' else '🟠' if expert1.confidence == 'Medium' else '🔴'}

{expert1.response}

---

## ✅ Expert 2: Verification Specialist

**Confidence:** {expert2.confidence} {'🟢' if expert2.confidence == 'High' else '🟠' if expert2.confidence == 'Medium' else '🔴'}

{expert2.response}

{'### ⚠️ Issues Found:' if expert2.issues_found else ''}
{''.join([f'- {issue}' + chr(10) for issue in expert2.issues_found])}

---

## 🏆 Expert 3: Final Authoritative Answer

**Confidence:** {expert3.confidence} {'🟢' if expert3.confidence == 'High' else '🟠' if expert3.confidence == 'Medium' else '🔴'}

{expert3.response}

{'### 📚 Sources:' if expert3.sources else ''}
{''.join([f'- {source}' + chr(10) for source in expert3.sources])}

---

> **⚠️ SAFETY WARNING:** Always consult your vehicle's owner manual and consider professional help for complex repairs. Improper maintenance can be dangerous.

---
"""

    display(Markdown(markdown_report))

print("✅ Markdown display function ready!")


✅ Markdown display function ready!


## 🚗 Ask Car Question

In [16]:
# Enter your car question
car_question = "How to use autopilot feature in tesla"

print(f"🚗 Question: {car_question}")
print("="*80)

🚗 Question: How to use autopilot feature in tesla


## Run 3-Expert Verification Process

In [17]:
# Run the verification workflow
print("\n🔄 Starting 3-Expert Verification Process...\n")
print("="*80)

result = verification_graph.invoke({
    "car_topic": car_question,
    "search_context": []
})

print("\n" + "="*80)
print("✅ Verification Complete!")
print("="*80)


🔄 Starting 3-Expert Verification Process...


🔧 Expert 1 (Primary Mechanic) researching...
      🔍 Searching for: How to use autopilot feature in tesla
      ✅ Found 3 sources
   ✅ Expert 1 complete (Confidence: High)

✅ Expert 2 (Verification Specialist) cross-checking...
      🔍 Searching for: How to use autopilot feature in tesla verification safety check correct procedure
      ✅ Found 3 sources
   ✅ Expert 2 complete (Confidence: Medium)
   ⚠️ Issues found: 2

🏆 Expert 3 (Conflict Resolver) reviewing...
   ✅ Expert 3 complete (Final Confidence: High)

✅ Verification Complete!


## Display Results in Beautiful Dashboard

In [25]:
# Display the dashboard
create_car_dashboard(
    car_question,
    result['expert1_response'],
    result['expert2_response'],
    result['expert3_response']
)

print("\n🎉 Scroll up to see your verified car maintenance guide!")


# 🚗 Car Knowledge Assistant

## How to use autopilot feature in tesla

**Status:** ⚠️ CONFLICTS RESOLVED

---

## 🔧 Expert 1: Primary Mechanic

**Confidence:** High 🟢

To use the Autopilot feature in a Tesla, follow these steps:

1. **Ensure Autopilot is Enabled**: Before using Autopilot, make sure it is enabled in your vehicle's settings. You can do this by going to the Autopilot settings in the car's touchscreen interface and ensuring that the Autopilot features you want to use are turned on.

2. **Engage Autopilot**:
   - For Tesla Model 3 and Model Y, pull down the gear changing lever twice in quick succession. You should hear a chime indicating that Autopilot is engaged [2].
   - For other models, the process might vary slightly, so refer to your specific model's manual.

3. **Using Autosteer and Traffic-Aware Cruise Control (TACC)**:
   - Autosteer keeps the vehicle centered in its lane and works in conjunction with TACC, which adjusts the car's speed based on the traffic ahead.
   - To engage Autosteer, ensure that the Autopilot Activation is set to Double Click if you want to use TACC independently [3].

4. **Navigate on Autopilot**:
   - This feature allows the vehicle to suggest and automatically change lanes to follow the navigation route. It is a BETA feature and requires configuration in the settings [3].

5. **Safety Warnings**:
   - Always keep your hands on the steering wheel and be prepared to take over at any time. Autopilot is not a fully autonomous driving system and requires driver supervision.
   - Pay attention to road conditions and traffic, as Autopilot may not handle all situations perfectly.

6. **Common Mistakes to Avoid**:
   - Do not assume the car can drive itself without supervision. Always be attentive and ready to intervene.
   - Ensure that the cameras and sensors are clean and unobstructed for optimal performance.

7. **Required Tools/Parts**: No additional tools or parts are required to use Autopilot, but ensure your Tesla software is up to date to access the latest features.

By following these steps and precautions, you can effectively use the Autopilot feature in your Tesla vehicle.

---

## ✅ Expert 2: Verification Specialist

**Confidence:** Medium 🟠

The information provided by Expert 1 on how to use the Autopilot feature in a Tesla is mostly accurate, but there are some discrepancies and additional details that need to be addressed:

1. **Engaging Autopilot**: Expert 1 correctly mentions that for Tesla Model 3 and Model Y, you pull down the gear changing lever twice to engage Autopilot. However, for vehicles without stocks, you engage Autosteer by double pressing the right scroll wheel, not the gear lever [1].

2. **Autopilot Activation Settings**: The mention of setting Autopilot Activation to Double Click for using TACC independently is not clearly supported by the sources. Instead, the double-click setting is related to engaging Autosteer using the right scroll button [1].

3. **Navigate on Autopilot**: Expert 1 correctly identifies this as a BETA feature that requires configuration in the settings.

4. **Safety Warnings**: The safety warnings provided are accurate and align with Tesla's guidelines, emphasizing the need for driver supervision.

5. **Common Mistakes**: The advice on keeping cameras and sensors clean is valid and important for optimal Autopilot performance.

6. **Required Tools/Parts**: No additional tools or parts are needed, which is correct.

Additional Information:
- It is important to ensure that the vehicle's software is up to date to access the latest Autopilot features and improvements.
- The Autopilot feature may not be available in all regions or on all roads, depending on local regulations and road conditions.

Overall, the response is mostly accurate but could benefit from clearer differentiation between models with and without stocks and more precise details on the settings for engaging Autopilot features.

### ⚠️ Issues Found:
- Engaging Autopilot for vehicles without stocks should be done by double pressing the right scroll wheel, not the gear lever.
- The mention of setting Autopilot Activation to Double Click for TACC is not clearly supported by sources.


---

## 🏆 Expert 3: Final Authoritative Answer

**Confidence:** High 🟢

To use the Autopilot feature in a Tesla, follow these comprehensive steps:

1. **Ensure Autopilot is Enabled**: Before using Autopilot, make sure it is enabled in your vehicle's settings. Access the Autopilot settings through the car's touchscreen interface and ensure the desired Autopilot features are activated.

2. **Engage Autopilot**:
   - For Tesla Model 3 and Model Y with stocks, pull down the gear changing lever twice in quick succession. You should hear a chime indicating that Autopilot is engaged.
   - For vehicles without stocks, engage Autosteer by double pressing the right scroll wheel on the steering wheel.

3. **Using Autosteer and Traffic-Aware Cruise Control (TACC)**:
   - Autosteer keeps the vehicle centered in its lane and works with TACC, which adjusts the car's speed based on the traffic ahead.
   - To engage Autosteer, ensure that the Autopilot Activation is set to Double Click if you want to use TACC independently.

4. **Navigate on Autopilot**:
   - This feature allows the vehicle to suggest and automatically change lanes to follow the navigation route. It is a BETA feature and requires configuration in the settings.

5. **Safety Warnings**:
   - Always keep your hands on the steering wheel and be prepared to take over at any time. Autopilot is not a fully autonomous driving system and requires driver supervision.
   - Pay attention to road conditions and traffic, as Autopilot may not handle all situations perfectly.

6. **Common Mistakes to Avoid**:
   - Do not assume the car can drive itself without supervision. Always be attentive and ready to intervene.
   - Ensure that the cameras and sensors are clean and unobstructed for optimal performance.

7. **Software Updates**: Ensure your Tesla software is up to date to access the latest Autopilot features and improvements.

8. **Regional Availability**: Be aware that the Autopilot feature may not be available in all regions or on all roads, depending on local regulations and road conditions.

By following these steps and precautions, you can effectively and safely use the Autopilot feature in your Tesla vehicle.

### 📚 Sources:
- https://www.findmyelectric.com/tesla-autopilot-ultimate-guide/
- https://tesloid.com/2023/06/07/how-do-i-turn-on-autopilot-in-a-tesla/?srsltid=AfmBOoou2PACSlqnK8pM3D_OGI1zOl2yUDtjMQ81sJomJGUHfINNJkVF
- https://www.tesla.com/ownersmanual/model3/en_au/GUID-20F2262F-CDF6-408E-A752-2AD9B0CC2FD6.html
- https://www.tesla.com/ownersmanual/model3/en_us/GUID-20F2262F-CDF6-408E-A752-2AD9B0CC2FD6.html
- https://www.facebook.com/groups/teslaownersaustralia/posts/732938918828186/


---

> **⚠️ SAFETY WARNING:** Always consult your vehicle's owner manual and consider professional help for complex repairs. Improper maintenance can be dangerous.

---



🎉 Scroll up to see your verified car maintenance guide!


---

## 🎯 Features

✅ **3-Expert Verification System**: Information → Verification → Conflict Resolution  
✅ **Real Web Search**: Current automotive information  
✅ **Conflict Detection**: Identifies discrepancies between experts  
✅ **Confidence Ratings**: Know how reliable the information is  
✅ **Source Citations**: All answers backed by sources  
✅ **Beautiful Dashboard**: Color-coded by verification status  
✅ **Safety Warnings**: Always prioritizes safety  

## 📚 Covers:

- 🔧 Maintenance & Repairs
- 🎨 Interior Customization  
- 🎛️ Features & Controls
- 📖 Owner's Manual Info
- 🛠️ DIY Repair Guides

---

*Car Knowledge Assistant - Triple-verified automotive information you can trust!* 🚗✨